In [58]:
import abc
import numpy as np
from collections import namedtuple
from scipy.stats import norm


# Inputs: 
#    F_t         = price of underlying
#    X_t         = strike
#    t           = time to expiration
#    v           = implied volatility
#    r           = risk free rate
#    q           = dividend payment
#    b           = cost of carry
# Outputs: 
#    value       = price of the option
#    delta       = 1st derivative of value w.r.t price of underlying
#    gamma       = 2nd derivative of value w.r.t price of underlying
#    theta       = 1st derivative of value w.r.t. time to expiration
#    vega        = 1st derivative of value w.r.t. implied volatility
#    rho         = 1st derivative of value w.r.t. risk free rates
Greeks = namedtuple("greeks", "value delta gamma theta vega rho")

In [59]:
class GeneralisedEuropeanBlackScholes(abc.ABC):
    
    def __init__(self, F_t, X_t, t, r, b, sigma):
        self.F_t = F_t
        self.X_t = X_t
        self.t = t
        self.r = r
        self.b = b
        self.sigma = sigma
        
        # Pre-calculations
        self.sqrt_t = np.sqrt(t)
        self.d1 = (np.log(F_t/X_t) + (b + np.square(sigma)/2) * t) / (sigma * self.sqrt_t)
        self.d2 = self.d1 - sigma * self.sqrt_t
        
    def call(self):
        # Unpack for readable equations
        F_t = self.F_t
        X_t = self.X_t
        t = self.t
        r = self.r
        b = self.b
        sigma = self.sigma
        d1 = self.d1
        d2 = self.d2
        sqrt_t = self.sqrt_t
        
        # Fair value price
        value = F_t * np.exp((b - r)*t) * norm.cdf(d1) - X_t * np.exp(-r*t) * norm.cdf(d2)

        # Greeks
        delta = np.exp((b - r)*t) * norm.cdf(d1)
        gamma = np.exp((b - r)*t) * norm.pdf(d1) / (F_t * sigma * sqrt_t)
        theta = -(F_t * sigma * np.exp((b - r)*t) * norm.pdf(d1)) / (2*sqrt_t) - (b - r) * F_t * np.exp((b - r)*t) * norm.cdf(d1) - r * X_t * np.exp(-r * t) * norm.cdf(d2)
        vega = np.exp((b - r)*t) * F_t * sqrt_t * norm.pdf(d1)
        rho = X_t * t * np.exp(-r*t) * norm.cdf(d2)
        
        return Greeks(value, delta, gamma, theta, vega, rho)
        
        
    def put(self):
        # Unpack for readable equations
        F_t = self.F_t
        X_t = self.X_t
        t = self.t
        r = self.r
        b = self.b
        sigma = self.sigma
        d1 = self.d1
        d2 = self.d2
        sqrt_t = self.sqrt_t
        
        # Fair value price
        value = X_t * np.exp(-r*t) * norm.cdf(-d2) - (F_t * np.exp((b - r)*t) * norm.cdf(-d1))
        
        # Greeks
        delta = -np.exp((b - r)*t) * norm.cdf(-d1)
        gamma = np.exp((b - r)*t) * norm.pdf(d1) / (F_t * sigma * sqrt_t)
        theta = -(F_t * sigma * np.exp((b - r)*t) * norm.pdf(d1)) / (2*sqrt_t) + (b - r) * F_t * np.exp((b - r)*t) * norm.cdf(-d1) + r * X_t * np.exp(-r*t) * norm.cdf(-d2)
        vega = np.exp((b - r)*t) * F_t * sqrt_t * norm.pdf(d1)
        rho = -X_t * t * np.exp(-r*t) * norm.cdf(-d2)

        return Greeks(value, delta, gamma, theta, vega, rho)

In [60]:
class Black76Eur(GeneralisedEuropeanBlackScholes):
    
    def __init__(self, F_t, X_t, t, r, sigma):
        b = 0
        super().__init__(F_t=F_t, X_t=X_t, t=t, r=r, b=b, sigma=sigma)

In [63]:
F_t = 50
X_t = 55
t = 1/12
r = 0.04
sigma = 0.20


black76 = Black76Eur(F_t=F_t, X_t=X_t, t=t, r=r, sigma=sigma)
black76.call(), black76.put()

(greeks(value=0.06213200829622734, delta=0.05223228374301364, gamma=0.03696551723344639, theta=-1.8457905813404705, vega=1.540229884726933, rho=0.2124568482378712),
 greeks(value=5.04549308856884, delta=-0.9444399323115097, gamma=0.03696551723344639, theta=-1.6464561381295657, vega=1.540229884726933, rho=-4.355624142012027))

In [ ]:
def euro_implied_vol_76(option_type, fs, x, t, r, cp):
    b = 0
    return _gbs_implied_vol(option_type, fs, x, t, r, b, cp)

In [64]:
scipy.optimize.newton

# _gbs_implied_vol wraps:
# _newton_implied_vol(_gbs, option_type, x, fs, t, b, r, cp, precision, max_steps)

# -> def _gbs()...

# _gbs is the equiv. of black76.call() and black76.put()

NameError: name 'scipy' is not defined